In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize
from jarvis.db.figshare import data
from jarvis.core.atoms import Atoms

def compute_density_matrix(C, num_electrons):
    num_occupied = num_electrons // 2
    return np.dot(C[:, :num_occupied], C[:, :num_occupied].T)

def compute_two_electron_integrals(P, k_point):
    G = np.zeros_like(P)
    size = P.shape[0]
    for p in range(size):
        for q in range(size):
            for r in range(size):
                for s in range(size):
                    phase_factor = np.exp(1j * k_point * (r - s))
                    two_electron_value = phase_factor.real / (1 + abs(p - q) + abs(r - s))
                    G[p, q] += P[r, s] * two_electron_value
    return G


/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def calculate_h_core(atoms, alpha_params, beta_params, gamma_params):
    """
    Генерация полуэмпирической H_core матрицы.
    alpha_params: словарь диагональных элементов (по элементу атома)
    beta_params: словарь внедиагональных элементов (по парам элементов)
    gamma_params: словарь дополнительных параметров
    """
    num_atoms = len(atoms.elements)
    H_core = np.zeros((num_atoms, num_atoms))
    coords = np.array(atoms.cart_coords)
    distances = squareform(pdist(coords))

    # Значения по умолчанию
    default_alpha = -5.0  # Значение для неизвестных элементов
    default_beta = -1.0   # Внедиагональное значение по умолчанию

    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                element = atoms.elements[i]
                H_core[i, j] = alpha_params.get(element, default_alpha)
            else:
                pair = tuple(sorted((atoms.elements[i], atoms.elements[j])))
                H_core[i, j] = beta_params.get(pair, default_beta) * np.exp(-distances[i, j])

    return H_core


In [3]:
def compute_fock_matrix(H_core, P, k_point):
    G = compute_two_electron_integrals(P, k_point)
    phase = np.exp(1j * k_point)
    return H_core * phase.real + G

def hartree_fock_pbc(S, H_core, num_electrons, k_points, max_iterations=50, convergence_threshold=1e-6):
    energy_total = 0.0
    P = np.zeros_like(H_core)
    S_inv_sqrt = np.linalg.inv(np.sqrt(S))
    
    for k in k_points:
        H_core_k = np.dot(S_inv_sqrt.T, np.dot(H_core, S_inv_sqrt))
        energy = 0.0
        for iteration in range(max_iterations):
            F = compute_fock_matrix(H_core_k, P, k)
            eigenvalues, eigenvectors = np.linalg.eigh(F)
            C = np.dot(S_inv_sqrt, eigenvectors)
            P_new = compute_density_matrix(C, num_electrons)
            energy_new = np.sum(P_new * (H_core + F))
            
            if abs(energy_new - energy) < convergence_threshold:
                break
            energy = energy_new
            P = P_new
        energy_total += energy
    return energy_total / len(k_points)


In [4]:

dft_2d_data = data('dft_2d')

# Параметры для H_core
alpha_params = {'C': -7.441409, 'B': -7.2, 'N': -6.8}  # Энергии на диагонали
beta_params = {('C', 'C'): -2.135484, ('B', 'N'): -2.0}  # Внедиагональные элементы
gamma_params = {} 


material_data = dft_2d_data[0]  # Берём первый материал для примера
if material_data:
    atoms = Atoms.from_dict(material_data['atoms'])
    S = np.eye(len(atoms.elements))  # Матрица перекрытий (единичная)
    
    H_core = calculate_h_core(atoms, alpha_params, beta_params, gamma_params)
    print("H_core матрица:\n", H_core)
else:
    print("Материал не найден.")


Загрузка данных JARVIS-DFT...
Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.
H_core матрица:
 [[-5.         -0.15320738 -0.03388534]
 [-0.15320738 -5.         -0.08392913]
 [-0.03388534 -0.08392913 -5.        ]]


In [5]:
k_points = [0.0, np.pi / 2, np.pi]
hf_energy = hartree_fock_pbc(S, H_core, num_electrons=2, k_points=k_points)
dft_energy = material_data.get('optb88vdw_total_energy', None)

print(f"Hartree-Fock энергия: {hf_energy:.6f} эВ")
print(f"JARVIS-DFT энергия: {dft_energy:.6f} эВ")


Hartree-Fock энергия: -4.429830 эВ
JARVIS-DFT энергия: -4.520940 эВ


In [6]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

def generate_features_extended(atoms, max_features=10):
    """
    Генерация расширенных признаков на основе расстояний, числа атомов и плотности структуры.
    """
    coords = np.array(atoms.cart_coords)
    distances = pdist(coords)
    distances = np.sort(distances)[:max_features]  # Берём max_features расстояний
    
    # Дополнение нулями до фиксированного размера
    features = np.pad(distances, (0, max_features - len(distances)), 'constant')

    # Дополнительные признаки
    num_atoms = len(atoms.elements)
    density = num_atoms / atoms.volume
    mean_distance = np.mean(distances) if len(distances) > 0 else 0

    features = np.append(features, [num_atoms, density, mean_distance])
    return features


In [7]:
from jarvis.db.figshare import data
from jarvis.core.atoms import Atoms
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

# Загрузка данных
dft_2d_data = data('dft_2d')

# Сбор данных
X, y = [], []
for material in dft_2d_data:
    atoms_data = material.get('atoms', None)
    energy_dft = material.get('optb88vdw_total_energy', None)

    if atoms_data and energy_dft:
        atoms = Atoms.from_dict(atoms_data)
        features = generate_features_extended(atoms)
        X.append(features)
        y.append(energy_dft)

X = np.array(X)
y = np.array(y)
print(f"Данные загружены: {X.shape[0]} материалов.")


Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.
Данные загружены: 1103 материалов.


In [8]:
# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели градиентного бустинга
print("Обучение модели...")
model = GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train)

# Оценка качества на тестовой выборке
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Средняя абсолютная ошибка на тесте: {mae:.6f} эВ")


Обучение модели...
Средняя абсолютная ошибка на тесте: 0.887124 эВ


In [15]:
# Предсказание для нового материала
new_material = dft_2d_data[0] 
atoms = Atoms.from_dict(new_material['atoms'])
features = generate_features_extended(atoms)

predicted_energy = model.predict([features])[0]
dft_energy = new_material['optb88vdw_total_energy']

print(f"Предсказанная энергия: {predicted_energy:.6f} эВ")
print(f"Энергия из JARVIS-DFT: {dft_energy:.6f} эВ")


Предсказанная энергия: -5.050642 эВ
Энергия из JARVIS-DFT: -4.520940 эВ
